In [1]:
import pandas as pd
import numpy as np

# Fact table

In [5]:
df_major = pd.read_csv('all_extracted_df_filtered_with_cats.csv')

In [6]:
print(df_major.columns.tolist())

['job_id', 'jobTitle', 'job_title_cat1', 'Category', 'GDJobTitle', 'Location', 'CompanyName', 'HqLocation', 'CompanySize', 'Industry', 'Sector', 'Star_rating', 'description']


In [7]:
df_major = df_major[[
 'job_id',
 'jobTitle',
 'Category',
 'Location',
 'CompanyName',
 'CompanySize',
 'Industry',
 'Sector',
 'Star_rating',
 'description'
]]

In [8]:
df_major.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40326 entries, 0 to 40325
Data columns (total 10 columns):
job_id         40326 non-null int64
jobTitle       40326 non-null object
Category       40326 non-null object
Location       40326 non-null object
CompanyName    40321 non-null object
CompanySize    35716 non-null object
Industry       40324 non-null object
Sector         35716 non-null object
Star_rating    35716 non-null float64
description    40326 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 3.1+ MB


In [9]:
# Size Cleaning
df_major.CompanySize = ['1-10000' if i =='10000--1' or i != i else '0-1' if i == '-1-0' else i for i in df_major.CompanySize]
df_major.CompanySize.value_counts()     

1-10000       15496
1001-5000      5780
51-200         4698
1-50           4286
201-500        4058
501-1000       2523
5001-10000     2220
0-1            1265
Name: CompanySize, dtype: int64

In [10]:
# Industry cleaning
df_major[df_major.Industry.isnull()]

,job_id,jobTitle,Category,Location,CompanyName,CompanySize,Industry,Sector,Star_rating,description
7872,3390001969,"Software Development Manager, AWS Security",Developer,"Baltimore, MD","Amazon Web Services, Inc.",51-200,NaN,Health Care,4.1,Bachelor's degree in Computer Science or relat...
34302,3389453534,Splunk Developer,Developer,"Seattle, WA",Recode Solutions,1-50,NaN,Information Technology,5.0,"Understand requirements, participating in crea..."


In [11]:
df_major[(df_major.Sector =='Health Care')&(df_major.CompanyName == 'Amazon Web Services, Inc.')]['Industry'].tail(2)

34850    Health Care Services & Hospitals
35238    Health Care Services & Hospitals
Name: Industry, dtype: object

In [12]:
df_major[(df_major.Sector =='Information Technology')&(df_major.CompanyName == 'Amazon Web Services, Inc.')]['Industry'].head(3)

7548                      IT Services
33713    Computer Hardware & Software
33730                     IT Services
Name: Industry, dtype: object

In [13]:
df_major['Industry'].loc[7872] = 'Health Care Services & Hospitals'
df_major['Industry'].loc[34302] = 'IT Services'

D:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
df_major[df_major.Industry.isnull()]

,job_id,jobTitle,Category,Location,CompanyName,CompanySize,Industry,Sector,Star_rating,description


In [15]:
# Sector cleaning

In [16]:
len(df_major.Sector.value_counts())

25

In [17]:
ml_sec_df = df_major[~df_major.Sector.isnull()][['Industry', 'Sector']]

In [18]:
# Clustering ML

In [19]:
df_major_ml = df_major
industry_ls = df_major.Industry.value_counts().index.tolist()

In [20]:
from nltk import ngrams
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

def text_preprocessing(ls):
    #ls_for_dn = google_soft.Google_name.value_counts().index.to_list()

    # remove punctuation
    punctuation_ls = '!"#$%&+\'()*,-./:;<=>?@[\\]^_`{|}~'
    ls_for_dn_transf = [st.translate(str.maketrans('', '', punctuation_ls)) for st in ls  if type(st) is str]
    print('Done1')
    # create tokens
    ls_for_dn_transf = [word_tokenize(st)for st in ls_for_dn_transf ]
    print('Done2')
    #remove stop words
    stop_words = stopwords.words('english')
    ls_for_dn_transf = [[st for st in ls if st.lower() not in stop_words] for ls in ls_for_dn_transf  ]
    print('Done3')
    # Back to String
    ls_for_dn_transf = [" ".join(ls) for ls in ls_for_dn_transf]
    print('Done4')
    return ls_for_dn_transf

In [21]:
industry_ls_clean = text_preprocessing (industry_ls)
df_major_ml = pd.merge (df_major_ml,pd.DataFrame({'Industry': industry_ls, 'Industry_clean': industry_ls_clean}), how= 'left', on ='Industry' )

Done1
Done2
Done3
Done4


In [22]:
#df_major_ml.head()

In [23]:
#Vectorization
#1st step
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=3000)
x = df_major_ml["Industry_clean"]
v.fit(x)
x_tfidf = v.transform(x)

In [24]:
#Model Testing
df_major_ml_train = df_major_ml[~df_major_ml.Sector.isnull()]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_major_ml_train.Industry, df_major_ml_train.Sector, test_size=0.2)

# Vectorization
x_tfidf_train = v.transform(X_train)
x_tfidf_test = v.transform(X_test)

In [25]:
## Training the model
from sklearn import svm
# Classifier - Algorithm - SVM
s = svm.LinearSVC(C=1.0)

# fit the training dataset on the classifier
s.fit(x_tfidf_train, y_train)# predict the labels on validation dataset
s_pred = s.predict(x_tfidf_test)

from sklearn.metrics import accuracy_score
# Use accuracy_score function to get the accuracy
print("Accuracy Score (SVM) -> ",accuracy_score(s_pred, y_test)*100)

Accuracy Score (SVM) ->  99.97200447928331


In [26]:
# Model Training and applying to Null Sector values
df_major_ml_train = df_major_ml[~df_major_ml.Sector.isnull()]
df_major_ml_test = df_major_ml[df_major_ml.Sector.isnull()]

X_train, X_test, y_train, y_test = df_major_ml_train.Industry, df_major_ml_test.Industry, df_major_ml_train.Sector, df_major_ml_test.Sector

In [27]:
# Vectorization
x_tfidf_train = v.transform(X_train)
x_tfidf_test = v.transform(X_test)

# Classifier - Algorithm - SVM
s = svm.LinearSVC(C=1.0)
# fit the training dataset on the classifier
s.fit(x_tfidf_train, y_train)# predict the labels on validation dataset
s_pred = s.predict(x_tfidf_test)

In [28]:
df_major_ml['Sector'][df_major_ml.Sector.isnull()] = s_pred

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
# checking the star raiting

In [30]:
df_major_ml = df_major_ml[['job_id', 'jobTitle', 'Category', 'Location', 'CompanyName', 'CompanySize', 'Industry', 'Sector', 'Star_rating', 'description', 'Industry_clean']]
df_major_ml = df_major_ml.sample(len(df_major_ml))
df_major_ml['Star_rating'].fillna(method = 'bfill', inplace = True)

import math, numpy
df_major_ml['Star_rating_round'] = [round(x) if math.isnan(x) is False else None  for x in  df_major_ml['Star_rating']]

In [31]:
df_major_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40326 entries, 38123 to 10774
Data columns (total 12 columns):
job_id               40326 non-null int64
jobTitle             40326 non-null object
Category             40326 non-null object
Location             40326 non-null object
CompanyName          40321 non-null object
CompanySize          40326 non-null object
Industry             40326 non-null object
Sector               40326 non-null object
Star_rating          40326 non-null float64
description          40326 non-null object
Industry_clean       40326 non-null object
Star_rating_round    40326 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 4.0+ MB


In [32]:
# Filling Company name
df_major_ml.CompanyName.fillna(value = 'NoName', inplace = True)

In [33]:
df_major_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40326 entries, 38123 to 10774
Data columns (total 12 columns):
job_id               40326 non-null int64
jobTitle             40326 non-null object
Category             40326 non-null object
Location             40326 non-null object
CompanyName          40326 non-null object
CompanySize          40326 non-null object
Industry             40326 non-null object
Sector               40326 non-null object
Star_rating          40326 non-null float64
description          40326 non-null object
Industry_clean       40326 non-null object
Star_rating_round    40326 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 4.0+ MB


In [34]:
print(df_major_ml.columns.tolist())

['job_id', 'jobTitle', 'Category', 'Location', 'CompanyName', 'CompanySize', 'Industry', 'Sector', 'Star_rating', 'description', 'Industry_clean', 'Star_rating_round']


In [35]:
#df_major_ml[['job_id', 'jobTitle', 'Category', 'Location', 'CompanyName', 'CompanySize', 'Industry', 'Sector', 'Star_rating', 'description', 'Industry_clean', 'Star_rating_round']]

In [36]:
# Final version of fact table
df_major_final = df_major_ml[['job_id', 'jobTitle', 'Category', 'CompanyName', 'CompanySize', 'Industry', 'Sector', 'Star_rating_round']]
df_major_final.columns = ['job_id', 'jobTitle', 'jobCategory', 'CompanyName', 'CompanySize','Industry', 'Sector', 'StarRating']
df_major_final.head()

,job_id,jobTitle,jobCategory,CompanyName,CompanySize,Industry,Sector,StarRating
38123,3216908031,ISR Data Analyst,Analyst,"Compass, Inc.",201-500,Colleges & Universities,Education,3
22855,3331510614,"Factory Service Environmental, Health, and Saf...",Other,GE Appliances,1-10000,Consumer Products Manufacturing,Manufacturing,4
35067,3390540112,Senior Software Engineer,Developer,Oracle,1-10000,Enterprise Software & Network Solutions,Information Technology,4
24302,3389740013,Financial Reporting- Technology Risk- Business...,Business Analytics,Wells Fargo,1-10000,Banks & Credit Unions,Finance,4
38853,3390405098,Track Management Engineering and Analysis,Consultant,Silver Bullet Solutions,1-10000,IT Services,Information Technology,3


In [22]:
df_major_final.to_csv('D:/Python Catalyst/Week 39 - Project work Selko_io/Analysis/Analysis 2nd version/Clean Data/fact_table.csv', index = False)

# Soft Name table

In [38]:
soft_names = pd.read_csv('google_orig_soft_names.csv')

In [44]:
soft_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115586 entries, 0 to 115585
Data columns (total 5 columns):
job_id                115586 non-null int64
soft_name             115586 non-null object
Google_name           115586 non-null object
Google_name_transf    115586 non-null object
soft_cat              104176 non-null object
dtypes: int64(1), object(4)
memory usage: 4.4+ MB


In [40]:
soft_names['soft_name'] = soft_names.soft_name.apply(lambda x: 'Not mentioned' if x !=x else x)

In [41]:
soft_names['Google_name'] =  soft_names.Google_name.apply(lambda x: 'Not mentioned' if x !=x else x)

In [42]:
soft_names['Google_name_transf'] =  soft_names.Google_name_transf.apply(lambda x: 'Not mentioned' if x !=x else x)

In [47]:
soft_name_up_ver = soft_names[['job_id','Google_name_transf', 'soft_cat']]
soft_name_up_ver.columns = ['job_id','SoftNames', 'SoftCatName']

In [238]:
soft_name_up_ver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115586 entries, 0 to 115585
Data columns (total 3 columns):
job_id         115586 non-null int64
SoftNames      115586 non-null object
SoftCatName    104176 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB


In [241]:
soft_name_up_ver['SoftNames']=soft_name_up_ver.SoftNames.apply(lambda x: x.lower().title())

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Clustering

In [71]:
all_tools = pd.read_csv('all_tools_df.csv')
all_tools.columns

Index(['big_data_tools', 'bi_tools', 'data_lang', 'more_data_tools',
       'data_warehouse'],
      dtype='object')

In [73]:
all_tools.data_lang[3] = 'C'
all_tools_up = all_tools[['bi_tools', 'data_lang','data_warehouse']]
all_tools_up.columns = ['Business Intelligence', 'Programming Language','Data Warehouse']
all_tools_up.head()

,Business Intelligence,Programming Language,Data Warehouse
0,TIBCO Spotfire,Python,Arm Treasure Data
1,SAP Crystal,R,Amazon Redshift
2,Tableau Desktop,SQL,Oracle Exadata Database Machine
3,Qlik Sense,C,Snowflake
4,TIBCO Jaspersoft,C++,SAP Business Warehouse


In [80]:
tools_and_cat = all_tools_up.stack().reset_index()
tools_and_cat = tools_and_cat[['level_1',0]]
tools_and_cat.columns = ['category', 'tool_name']

In [121]:
from sklearn.metrics.pairwise import cosine_similarity as cosi
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import clear_output
import string

In [208]:
# creating Categories
def soft_categorizer(ls_soft, tools_ls):
    cat_ls = []
    tools_ls_lower = [i.lower() for i in tools_ls]
    for i in range(len(ls_soft)):
        temp_ls =[ls_soft[i].lower()]+tools_ls_lower
        #vectorize
        
        string_length = ls_soft[i].translate(str.maketrans('', '', string.punctuation))
        if len(string_length)<=1:
            v1 =  CountVectorizer(analyzer = 'char').fit_transform(temp_ls).toarray()
            
            max_sim = max(cosi(v1)[0][1:])
            if max_sim >0.7:
                ind_num_of_job_name = cosi(v1)[0][1:].tolist().index(max_sim)
                assigned_label = tools_and_cat.category[ind_num_of_job_name]
                cat_ls.append(assigned_label)
            
            elif max_sim <0.4:
                cat_ls.append('Other')

            else:
                cat_ls.append(None)
        
        else:
            v1 =  CountVectorizer(analyzer = 'word').fit_transform(temp_ls).toarray()
            
        #max similarity level
            max_sim = max(cosi(v1)[0][1:])

            # conditional statement 
            if max_sim >0.79:
                ind_num_of_job_name = cosi(v1)[0][1:].tolist().index(max_sim)
                assigned_label = tools_and_cat.category[ind_num_of_job_name]
                cat_ls.append(assigned_label)

            elif max_sim <0.4:
                cat_ls.append('Other')

            else:
                cat_ls.append(None)
        clear_output()
        print('Completed:',i+1)
        print('Left:',len(ls_soft) - (i+1))
    return (cat_ls)

In [209]:
#ls_soft
ls_soft = soft_name_up_ver.SoftNames.value_counts().index.tolist()
tools_ls = tools_and_cat.tool_name.tolist()
categories_ls = soft_categorizer(ls_soft = ls_soft, tools_ls = tools_ls)

Completed: 4737
Left: 0


In [210]:
len(categories_ls)

4737

In [128]:
#pd.DataFrame({'SoftNames':ls_soft, 'SoftCategory':categories_ls})
#all_tools_up['Business Intelligence']
ls_t = ['Power Bi','Microsoft Power Bi', 'Microsoft Office','Microsoft','Tableau Server','Tableau','S','Tableau Desktop','S']
ls_t = ['CC','CC']
v1_t =  CountVectorizer(analyzer = 'word').fit_transform(ls_t).toarray()
cosi(v1_t)[4]


array([0.        , 0.        , 0.        , 0.        , 1.        ,
       0.70710678, 0.        , 0.5       , 0.        ])

In [157]:
#all_tools_up['Business Intelligence'].dropna()

In [211]:
#all_tools_up['Programming Language']
df_soft_with_cat = pd.merge(soft_name_up_ver, pd.DataFrame({'SoftNames':ls_soft, 'SoftCategory':categories_ls}), how= 'left', on ='SoftNames')

In [236]:
sum(df_soft_with_cat.SoftCategory.value_counts())

61391

In [233]:
# Business Intelligence      
# Data Warehouse             
# Programming Language  

df_s = pd.DataFrame({'SoftNames':ls_soft, 'SoftCategory':categories_ls})
df_s[df_s.SoftCategory == 'Programming Language']

,SoftNames,SoftCategory
2,SQL,Programming Language
4,Python,Programming Language
6,Java,Programming Language
7,JavaScript,Programming Language
11,R,Programming Language
16,C++,Programming Language
20,Scala,Programming Language
21,C,Programming Language
38,MATLAB,Programming Language
72,Swift,Programming Language


In [187]:
'Qlik View'.lower().title()

'Qlik View'

In [191]:
len(df_soft_with_cat)

115586

In [212]:
df_soft_with_cat.head(3)

,job_id,SoftNames,SoftCatName,SoftCategory
0,3394200362,Microsoft Office,Microsoft,None
1,3395019069,Not mentioned,NaN,Other
2,3310569642,Amazon Web Services,Amazon,None


In [213]:
soft_to_clean = df_soft_with_cat.SoftNames.value_counts().index.tolist()
soft_clean_ls = text_preprocessing(soft_to_clean)

Done1
Done2
Done3
Done4


In [271]:
df_soft_with_cat_svm = pd.merge(df_soft_with_cat,pd.DataFrame({'SoftNames':soft_to_clean,'SoftNames_clean':soft_clean_ls}), how= 'left', on='SoftNames')
print(len(df_soft_with_cat_svm))
df_soft_with_cat_svm.head(3)
#df_soft_with_cat_svm[(~df_soft_with_cat_svm.SoftCategory.isnull()) & (df_soft_with_cat_svm.SoftCategory != 'Programming Language')]
df_soft_with_cat_svm.SoftCategory.value_counts()

115586


Other                    33123
Programming Language     25840
Business Intelligence     1779
Data Warehouse             649
Name: SoftCategory, dtype: int64

In [258]:
#Vectorization
#1st step
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=3000)
x = df_soft_with_cat_svm[df_soft_with_cat_svm.SoftCategory != 'Programming Language']["SoftNames_clean"]
v.fit(x)
#x_tfidf = v.transform(x)
print("Tfid fitted")

Tfid fitted


In [259]:
len(x)

89746

In [261]:
len(df_major_ml_train)

35551

In [260]:
#Model Testing
df_major_ml_train = df_soft_with_cat_svm[(~df_soft_with_cat_svm.SoftCategory.isnull()) & (df_soft_with_cat_svm.SoftCategory != 'Programming Language')]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_major_ml_train.SoftNames_clean, df_major_ml_train.SoftCategory, test_size=0.2)

# Vectorization
x_tfidf_train = v.transform(X_train)
x_tfidf_test = v.transform(X_test)

In [262]:
## Training the model
from sklearn import svm
# Classifier - Algorithm - SVM
s = svm.LinearSVC(C=1.0)

# fit the training dataset on the classifier
s.fit(x_tfidf_train, y_train)# predict the labels on validation dataset
s_pred = s.predict(x_tfidf_test)

from sklearn.metrics import accuracy_score
# Use accuracy_score function to get the accuracy
print("Accuracy Score (SVM) -> ",accuracy_score(s_pred, y_test)*100)

Accuracy Score (SVM) ->  99.94374912108002


In [264]:
#df_soft_with_cat_svm[(df_soft_with_cat_svm.SoftCategory.isnull()) & (df_soft_with_cat_svm.SoftCategory != 'Programming Language')]


In [265]:
# Model Training and applying to Null Sector values
df_major_ml_train = df_soft_with_cat_svm[(~df_soft_with_cat_svm.SoftCategory.isnull()) & (df_soft_with_cat_svm.SoftCategory != 'Programming Language')]
df_major_ml_test = df_soft_with_cat_svm[(df_soft_with_cat_svm.SoftCategory.isnull()) & (df_soft_with_cat_svm.SoftCategory != 'Programming Language')]

X_train, X_test, y_train, y_test = df_major_ml_train.SoftNames_clean, df_major_ml_test.SoftNames_clean, df_major_ml_train.SoftCategory, df_major_ml_test.SoftCategory

In [266]:
# Vectorization
x_tfidf_train = v.transform(X_train)
x_tfidf_test = v.transform(X_test)

# Classifier - Algorithm - SVM
s = svm.LinearSVC(C=1.0)
# fit the training dataset on the classifier
s.fit(x_tfidf_train, y_train)# predict the labels on validation dataset
s_pred = s.predict(x_tfidf_test)

In [267]:
df_soft_with_cat_svm_ml_done = df_soft_with_cat_svm
df_soft_with_cat_svm_ml_done['SoftCategory'][df_soft_with_cat_svm_ml_done.SoftCategory.isnull()] = s_pred

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [275]:
print(df_soft_with_cat_svm.SoftCategory.value_counts())
print()
print(df_soft_with_cat_svm_ml_done.SoftCategory.value_counts())

Other                    33123
Programming Language     25840
Business Intelligence     1779
Data Warehouse             649
Name: SoftCategory, dtype: int64

Other                    82095
Programming Language     25840
Business Intelligence     6346
Data Warehouse            1305
Name: SoftCategory, dtype: int64


In [268]:
df_soft_with_cat_svm_ml_done[df_soft_with_cat_svm_ml_done.SoftCategory == 'Programming Language']['SoftNames'].value_counts()

SQL           7399
Python        6043
Java          3540
JavaScript    3005
R             2065
C++           1426
Scala          967
C              846
MATLAB         409
Swift          134
Javascript       3
Sql              2
Matlab           1
Name: SoftNames, dtype: int64

In [288]:
grouped_cat = df_soft_with_cat_svm_ml_done.groupby(['SoftCategory', 'SoftNames'])
df_soft_with_cat_svm_ml_done[df_soft_with_cat_svm_ml_done.SoftCategory == 'Data Warehouse' ]['SoftNames'].value_counts()

DB2                                      531
Snowflake                                307
Amazon Redshift                          145
Vertica                                  100
SAP Cloud Platform                        48
SAP BW                                    38
Modern Data Warehouse                     34
Exadata Database Machine                  14
IBM Db2 Warehouse                         12
MemSQL                                    12
Integrated Data Warehouse                  8
DB2 SQL                                    7
SAP NetWeaver Business Warehouse           6
Azure Data Warehouse                       6
Db2                                        5
IBM Db2                                    5
Unified Data Analytics Platform            4
Data Warehouse DBA                         3
Oracle Managed Cloud Database Service      3
Databricks Unified Analytics Platform      2
Redshift                                   2
Cloud Data Warehouse Software              2
Oracle Aut

In [286]:
import re
re.findall('.[^A-Z]*', 'QlikView Dashboards')

['Qlik', 'View ', 'Dashboards']

In [296]:
#df_soft_with_cat_svm_ml_done['SoftCatName_new']
df_soft_with_cat_svm_ml_done['SoftVendor'] = df_soft_with_cat_svm_ml_done.apply (lambda x: x['SoftNames'].split()[0] if len(x['SoftNames'].split()[0])>0 and
                                   x['SoftNames'] != 'Not mentioned' and
                                   x['SoftCategory'] != 'Programming Language' else None, axis = 1)

In [299]:
df_soft_with_cat_svm_ml_done.SoftVendor.value_counts().head(10)

Microsoft     27235
Amazon         5617
Apache         4077
Oracle         4020
Tableau        2625
Google         2116
SAS            1742
SAP            1737
Salesforce     1581
IBM            1291
Name: SoftVendor, dtype: int64

In [300]:
df_soft_with_cat_svm_ml_done.head()

,job_id,SoftNames,SoftCatName,SoftCategory,SoftNames_clean,SoftVendor
0,3394200362,Microsoft Office,Microsoft,Other,Microsoft Office,Microsoft
1,3395019069,Not mentioned,NaN,Other,mentioned,None
2,3310569642,Amazon Web Services,Amazon,Other,Amazon Web Services,Amazon
3,3310569642,Tableau,Tableau,Business Intelligence,Tableau,Tableau
4,3310569642,Java,Java,Programming Language,Java,None


In [306]:
final_soft_df  = df_soft_with_cat_svm_ml_done[['job_id', 'SoftNames', 'SoftVendor', 'SoftCategory']]
final_soft_df.columns = ['job_id', 'SoftName', 'SoftVendor', 'SoftCategory']
final_soft_df['SoftCategory'] = final_soft_df.apply(lambda x: None if x['SoftName'] == 'Not mentioned' 
                                                    else x['SoftCategory'], axis = 1)


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [309]:
final_soft_df.to_csv('D:/Python Catalyst/Week 39 - Project work Selko_io/Analysis/Analysis 2nd version/Clean Data/software_names.csv')

In [2]:
soft_df = pd.read_csv('D:/Python Catalyst/Week 39 - Project work Selko_io/Analysis/Analysis 2nd version/Clean Data/software_names.csv')

In [5]:
soft_df = soft_df[['job_id', 'SoftName', 'SoftVendor', 'SoftCategory']]


In [7]:
soft_df.SoftCategory.value_counts()

Other                    70685
Programming Language     25840
Business Intelligence     6346
Data Warehouse            1305
Name: SoftCategory, dtype: int64

In [8]:
soft_df.columns

Index(['job_id', 'SoftName', 'SoftVendor', 'SoftCategory'], dtype='object')

In [10]:
soft_df[soft_df.SoftCategory == 'Business Intelligence']['SoftName'].value_counts().head()

Tableau               2493
SAP                    841
Microsoft Power BI     719
Google Analytics       549
Looker                 194
Name: SoftName, dtype: int64

In [13]:
soft_df['SoftCategory'][(soft_df.SoftName =='SAP')&(soft_df.SoftCategory == 'Business Intelligence') ] = 'Other'

D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Programs\Anaconda\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
D:\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
soft_df['SoftCategory'][(soft_df.SoftName =='SAP')&(soft_df.SoftCategory == 'Business Intelligence') ]

Series([], Name: SoftCategory, dtype: object)

In [15]:
soft_df[soft_df.SoftCategory == 'Business Intelligence']['SoftName'].value_counts().head()

Tableau               2493
Microsoft Power BI     719
Google Analytics       549
Looker                 194
KEEN                   141
Name: SoftName, dtype: int64

In [16]:
soft_df.to_csv('D:/Python Catalyst/Week 39 - Project work Selko_io/Analysis/Analysis 2nd version/Clean Data/software_names.csv', index = False)

# Appending Geo Keys 

In [5]:
fact = pd.read_csv('Clean Data/fact_table.csv')
soft = pd.read_csv('Clean Data/software_names.csv')
location = pd.read_csv('Clean Data/google_location.csv')
loc_con = pd.read_csv('Clean Data/job_and_loc_id_temp.csv')

In [6]:
loc_con.columns

Index(['job_id', 'location_id'], dtype='object')

In [7]:
fact_up = pd.merge(fact,loc_con, how = 'left', on ='job_id' )
soft_up = pd.merge(soft,loc_con, how = 'left', on ='job_id' )
location_up = pd.merge(loc_con,location, how = 'left', on ='location_id' )

In [15]:
fact_up.to_csv('Clean Data/fact_table.csv', index = False)
soft_up.to_csv('Clean Data/software_names.csv', index = False)
location_up.to_csv('Clean Data/google_location.csv', index = False)

In [14]:
print (fact_up.columns,'\n\n',soft_up.columns,'\n\n',location_up.columns  )

Index(['job_id', 'jobTitle', 'jobCategory', 'CompanyName', 'CompanySize',
       'Industry', 'Sector', 'StarRating', 'location_id'],
      dtype='object') 

 Index(['job_id', 'SoftName', 'SoftVendor', 'SoftCategory', 'location_id'], dtype='object') 

 Index(['job_id', 'location_id', 'city', 'state', 'state_short', 'country',
       'lat', 'lon', 'bbox'],
      dtype='object')


In [31]:
soft = pd.read_csv('Clean Data/software_names.csv')

In [32]:
soft.columns

Index(['job_id', 'SoftName', 'SoftVendor', 'SoftCategory', 'location_id'], dtype='object')

In [33]:
soft[soft.SoftCategory == 'Business Intelligence']['SoftName'].value_counts().head(10)

Tableau                    2493
Microsoft Power BI          719
Google Analytics            549
Looker                      194
KEEN                        141
Power BI                    120
Azure Synapse Analytics     100
IBM Cloud                    73
Pentaho                      65
IBM Watson Health            57
Name: SoftName, dtype: int64

In [34]:
soft['SoftName'] = soft['SoftName'].apply(lambda x: 'Microsoft Power BI' if x== 'Power BI' else x)

In [35]:
soft.SoftCategory.value_counts()

Other                    71526
Programming Language     25840
Business Intelligence     5505
Data Warehouse            1305
Name: SoftCategory, dtype: int64

In [36]:
soft['SoftCategory']=soft.apply(lambda x: 'Other' if x['SoftName'] in ['IBM Cloud', 'IBM Watson Health'] else x['SoftCategory'], axis = 1) 

In [37]:
soft[soft.SoftCategory == 'Business Intelligence']['SoftName'].value_counts().head(10)

Tableau                                                  2493
Microsoft Power BI                                        839
Google Analytics                                          549
Looker                                                    194
KEEN                                                      141
Azure Synapse Analytics                                   100
Pentaho                                                    65
PRISM                                                      55
Board                                                      51
Oracle Business Intelligence Suite Enterprise Edition      50
Name: SoftName, dtype: int64

In [38]:
soft["Count1"] =1

In [39]:
soft.head()

,job_id,SoftName,SoftVendor,SoftCategory,location_id,Count1
0,3394200362,Microsoft Office,Microsoft,Other,2585522626,1
1,3395019069,Not mentioned,NaN,NaN,2585522626,1
2,3310569642,Amazon Web Services,Amazon,Other,2585522626,1
3,3310569642,Tableau,Tableau,Business Intelligence,2585522626,1
4,3310569642,Java,NaN,Programming Language,2585522626,1


In [40]:
soft.to_csv('Clean Data/software_names.csv', index = False)